# 1. Imports

In [1]:
from sklearn.decomposition import PCA
import matplotlib as mpl 
import numpy as np
import os
import pandas as pd
import pickle

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import f1_score
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import StandardScaler


from codes.utils import stratified_train_test_group_kfold
from codes.utils import model_test_classification

from codes.classification_codes import svc_gridsearch



0.10.1


# 2. Data

In [2]:


with open("Data/MoS2_Analysis_Augmented_trained", "rb") as fp:   # Unpickling
    MoS2_Proj1_Class_Data = pickle.load(fp)

#features = MoS2_ImageNet_100_data[0]
df = pd.DataFrame(MoS2_Proj1_Class_Data)
#T_dict= {900:0, 950:1, 1000:2}
T_target = np.array([item for item in df['T']])

features = np.array(list(df['ImageNet']))
sampleId = [item for item in df['sampleId']]

X = features
Y = np.array(T_target)
groups = np.array(sampleId)


train_val_groups, train_val_X, train_val_Y, test_X, test_Y = stratified_train_test_group_kfold(X, Y, groups, n_splits=10, test_fold=0)

scaler = StandardScaler().fit(X)
X = scaler.transform(X)
#val_X = scaler.transform(val_X)

Y = Y.flatten()



print(train_X.shape)
print(train_Y.shape)
print(val_X.shape)
print(val_Y.shape)

# 3. Runing Cross-Validation

In [8]:
Ce = [1.0, 0.8, 0.2]
Kernel= ['poly']#, 'rbf', 'sigmoid']#, 'precomputed']
Degree = [i for i in range(2, 8)]
Gamma = ['scale', 'auto', 0.1, 0.2, 0.3, 0.34]# or float, default=’scale’ 0.34 is the best
Coef0 = [0.8, 0.2, 0.05, 0.0]

Max_iter = [90, 100, 120, -1]
variables = [Ce, Kernel, Degree, Gamma, Coef0, Max_iter]


def cross_10_folds_svc(X, Y):
    
    best_train = []
    best_val = []
    best_variables = []
    for fold in range(10):
        
        model_path = f'classification/ImageNet/aug2/SVC_{fold}_model.sav'
        group, train_X, train_Y, val_X, val_Y = stratified_train_test_group_kfold(X, Y, groups, n_splits=10, test_fold=fold)
          
        
        #train_Y = [item[0] for item in train_Y]
        #val_Y = [item[0] for item in val_Y]
        
        variables_best, performance_best = svc_gridsearch(train_X, train_Y, val_X, val_Y, Ce, Kernel, Degree, Gamma, Coef0, Max_iter, fold, model_path)
        best_train.append(performance_best['accuracy_train'])
        best_val.append(performance_best['accuracy_val'])
        best_variables.append(variables_best)
        print(f'fold: {fold} done!')
    return best_train, best_val, best_variables


best_train, best_val, best_variables = cross_10_folds_svc(X, Y)
print(best_train)
print(best_val)
print(f'train_acc_mean: {np.mean(best_train) :.3f}, std: {np.std(best_train) :.3f}, val_acc_mean: {np.mean(best_val):.3f}, std: {np.std(best_val):.3f}')

<class 'generator'>
best_train_acc: 0.4889589905362776, best_val_acc: 0.5342465753424658


/noether/s0/iam5249/anaconda3/envs/torchvision/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=90).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/noether/s0/iam5249/anaconda3/envs/torchvision/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/noether/s0/iam5249/anaconda3/envs/torchvision/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=120).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/noether/s0/iam5249/anaconda3/envs/torchvision/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=90).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn

best_train_acc: 0.9858044164037855, best_val_acc: 0.6575342465753424


/noether/s0/iam5249/anaconda3/envs/torchvision/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=120).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/noether/s0/iam5249/anaconda3/envs/torchvision/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=90).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/noether/s0/iam5249/anaconda3/envs/torchvision/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/noether/s0/iam5249/anaconda3/envs/torchvision/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=120).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.war

fold: 0 done!
<class 'generator'>
best_train_acc: 0.5150078988941548, best_val_acc: 0.44594594594594594
best_train_acc: 0.5418641390205371, best_val_acc: 0.4594594594594595
best_train_acc: 0.5466034755134281, best_val_acc: 0.5


...

/noether/s0/iam5249/anaconda3/envs/torchvision/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/noether/s0/iam5249/anaconda3/envs/torchvision/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=120).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/noether/s0/iam5249/anaconda3/envs/torchvision/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=90).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/noether/s0/iam5249/anaconda3/envs/torchvision/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.war

best_train_acc: 1.0, best_val_acc: 0.6212121212121212


/noether/s0/iam5249/anaconda3/envs/torchvision/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=90).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/noether/s0/iam5249/anaconda3/envs/torchvision/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/noether/s0/iam5249/anaconda3/envs/torchvision/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=120).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/noether/s0/iam5249/anaconda3/envs/torchvision/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=90).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn

fold: 9 done!
[0.9858044164037855, 0.5102685624012638, 1.0, 1.0, 1.0, 0.5466034755134281, 0.5647425897035881, 1.0, 0.5101404056162246, 1.0]
[0.6575342465753424, 0.6081081081081081, 0.5675675675675675, 0.5405405405405406, 0.5945945945945946, 0.6216216216216216, 0.7272727272727273, 0.6060606060606061, 0.5909090909090909, 0.6212121212121212]
train_acc_mean: 0.812, std: 0.228, val_acc_mean: 0.614, std: 0.048


/noether/s0/iam5249/anaconda3/envs/torchvision/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=90).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/noether/s0/iam5249/anaconda3/envs/torchvision/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/noether/s0/iam5249/anaconda3/envs/torchvision/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=120).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/noether/s0/iam5249/anaconda3/envs/torchvision/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=90).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn

In [9]:
import numpy as np
from sklearn.metrics import f1_score


def f1score_fn(X, Y):

    F1_score_train = []
    F1_score_val = []
    for fold in range(10):
        
        group, train_X, train_Y, val_X, val_Y = stratified_train_test_group_kfold(X, Y, groups, n_splits=10, test_fold=fold)
        #train_X, train_Y = oversample.fit_resample(train_X, train_Y)
        model_path = f'classification/ImageNet/aug2/SVC_{fold}_model.sav'
        
        PATH = os.path.join('Model', model_path)
        
        loaded_model = pickle.load(open(PATH, 'rb'))

        pred_val_Y = loaded_model.predict(val_X)
        pred_train_Y = loaded_model.predict(train_X)

        f_score_train = f1_score(train_Y, pred_train_Y, average='macro')#'weighted')
        f_score_val = f1_score(val_Y, pred_val_Y, average='macro')#'weighted')

        F1_score_train.append(f_score_train)
        F1_score_val.append(f_score_val)
    
    return F1_score_train, F1_score_val



F1_score_train, F1_score_val = f1score_fn(X, Y)


print(f'f1_train: {np.mean(F1_score_train)}, std: {np.std(F1_score_train)}, f1_val: {np.mean(F1_score_val)}, std: {np.std(F1_score_val)}')


<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
f1_train: 0.806137433138694, std: 0.2357557608693939, f1_val: 0.6003526602378385, std: 0.051679640844984505
